In [76]:
#Importando bibliotecas e abrindo arquivos necessários
import pandas as pd
import numpy as np
from Análise_de_Correlação import analiseCorr
from Normalização import normalizaDf
df= pd.read_excel("input_acoes_NotasFundamentei_20200517.xlsx")
abas= pd.ExcelFile("input_planilhaCanalHolder_20200211.xlsx")
MA=abas.parse('Múltiplos Ações')
FA=abas.parse('Fundamentos Ações')

In [77]:
#Fazendo alterações no arquivo, removendo espaços nos nomes das colunas
MA.columns = [str(c).replace(' ', '_') for c in MA.columns]
FA.columns = [str(c).replace(' ', '_') for c in FA.columns]
df.columns = [str(c).replace(' ', '_') for c in df.columns]

In [3]:
#Join das tabelas "Fundamentos ações" e "NotasCanalFundamentei" e um merge entre as duas primeiras abas do arquivo input_planilhaCanalHolder_20200211.xlsx
MA=MA.join(df.set_index("Código_de_Neg."),on="Código_de_Neg.")
uniaoAbas = pd.merge(MA, FA, on="Nome", how='outer', suffixes=('', '_y'))
uniaoAbas = uniaoAbas.drop(list(uniaoAbas.filter(regex='_y$')), axis=1)
#Remoção de uma coluna inútil
uniaoAbas.drop("Unnamed:_0",axis=1,inplace=True)

In [60]:
dfCompleto=uniaoAbas.replace("-",np.nan)

In [61]:
dfCompleto.dropna(inplace=True)

In [62]:
dfNormalizado=normalizaDf(dfCompleto)

In [63]:
#Chamando a função principal
analiseCorrelação=analiseCorr("NotaFundamentei20200201",dfNormalizado)[2:]

In [64]:
def calculoAutomatizadoScore(analise,dFrame):
    count=0
    for peso in analise.values:
        if peso>0.236:
            print(peso)
            print(analise.axes[0][count])                #score+=dFrame[analise.axes[0][count]][linha]*peso
        if peso<-0.236:
            print(peso)
            print(analise.axes[0][count])        
        count+=1
    return dFrame
""" Paramêtros:
        analise: Analise de Correlação feita
        dataFrame: DataFrame contendo as empresas a serem rankeadas
    Return: O mesmo DataFrame do parâmetro, agora contendo a coluna Score """

' Paramêtros:\n        analise: Analise de Correlação feita\n        dataFrame: DataFrame contendo as empresas a serem rankeadas\n    Return: O mesmo DataFrame do parâmetro, agora contendo a coluna Score '

In [65]:
#Descobrindo quais são os principais atributos responsáveis pelo aumento de Score das empresas
testeAutomatizado=calculoAutomatizadoScore(analiseCorrelação,dfNormalizado)

0.3
Lucro_Líquido
0.254
Fluxo_de_Caixa_de_Financiamentos_(FCF)
0.244
Crescimento_Médio_Anual
0.238
Cotação
-0.237
Dívida_Bruta
-0.239
Payout
-0.33
Caixa_Líquido


In [66]:
#Tratamento feito para trabalhar com atributos que tiveram valores negativos na analise de correlação,
#basicamente, quanto menor eles forem, maior serão o Score da empresa
dfNormalizado["Dívida_Bruta"]=1-dfNormalizado["Dívida_Bruta"]
dfNormalizado["Payout"]=1-dfNormalizado["Payout"]
dfNormalizado["Caixa_Líquido"]=1-dfNormalizado["Caixa_Líquido"]

In [67]:
#Criação da coluna "Score", calculada por meio de uma média ponderada com os atributos mais importantes,
#os pesos estão relacionados aos seus respectivos valores na analise de correlação, exemplo: atributos
#com 0,3 na analise terão peso 3 na média ponderada
dfNormalizado["Score"]=((dfNormalizado["Lucro_Líquido"]*3)+
(dfNormalizado["Fluxo_de_Caixa_de_Financiamentos_(FCF)"]*2)+
(dfNormalizado["Crescimento_Médio_Anual"]*2)+
(dfNormalizado["Cotação"]*2)+(dfNormalizado["Dívida_Bruta"]*2)+
(dfNormalizado["Payout"]*2)+(dfNormalizado["Caixa_Líquido"]*3))/10

In [68]:
dfNormalizado["Score"]=((dfNormalizado["Score"]-dfNormalizado["Score"].min())/(dfNormalizado["Score"].max()-dfNormalizado["Score"].min()))*10

In [69]:
dfNormalizado["Score"]=dfNormalizado["Score"].round(1)

In [72]:
dfNormalizado["Nome"]=dfCompleto["Nome"]

In [75]:
dfNormalizado.sort_values("Score",ascending=False)[["Nome","Score"]].head(5)

,Nome,Score
31,Ambev S/A,10.0
278,RaiaDrogasil,9.9
232,Magaz Luiza,8.9
356,Totvs,8.7
218,Localiza,8.5


In [74]:
df[df["Código_de_Neg."]=="ABEV3"]

,Código_de_Neg.,NotaFundamentei,NotaFundamentei20200201
6,ABEV3,4.6,4.5
